#Домашнє завдання до тем apply(), groupby(), pivot_table()

В цьому домашньому завданні продовжуємо працювати з набором даних `supermarket_sales.csv`.

0. Імпортуйте бібліотеку pandas та зчитайте дані у змінну `df` типу `pandas.DataFrame`.

In [24]:
import pandas as pd

In [25]:
data_path = "./data/supermarket_sales.csv"
df = pd.read_csv(data_path)
df.head(1)

,Invoice ID,Branch,City,Customer type,Gender,Product line,Unit price,Quantity,Tax 5%,Total,Date,Time,Payment,cogs,gross margin percentage,gross income,Rating
0,750-67-8428,A,Yangon,Member,Female,Health and beauty,74.69,7,26.1415,548.9715,1/5/2019,13:08,Ewallet,522.83,4.761905,26.1415,9.1


1. Дослідимо, який філіал супермаркету ('Branch') є найприбутковішим. Для цього знайдіть сумарний прибуток за кожним філіалом і виявіть, який філіал має найвищий.

In [26]:
branch_profit = df.groupby('Branch')['gross income'].sum().reset_index(name='Total Profit')
branch_profit_sorted = branch_profit.sort_values(by='Total Profit', ascending=False)
print(f"Філіал з найвищим сумарним прибутком:\n{branch_profit_sorted.head(1)}")

Філіал з найвищим сумарним прибутком:
  Branch  Total Profit
2      C     5265.1765


2. В якому місті знайходиться філіал з найвищим прибутком? Може в тому місці нам розмітисти ще один магазин.  
Знайдіть відповідь за допомогою функціоналу Pandas.

In [27]:
city_of_branch = df[
    df['Branch'] == branch_profit_sorted.iloc[0]['Branch']
]['City'].unique()[0]
print(f"Місто, де знаходиться найприбутковіший філіал '{branch_profit_sorted.iloc[0]['Branch']}': {city_of_branch}")

Місто, де знаходиться найприбутковіший філіал 'C': Naypyitaw


3.1. Створіть зводну таблицю, яка покаже, скільки покупок (інвойсів) було зроблено в кожній з філій (`Branch`) за різними категоріями товарів. Запишіть таблицю в змінну `invoices_by_category` і виведіть змінну на екран.
Ця таблиця допоможе проаналізувати, в якій філії купують найбільше товарів кожної з категорій.

In [28]:
invoices_by_category = df.pivot_table(index='Branch', columns='Product line', values='Invoice ID', aggfunc='count', fill_value=0)
invoices_by_category

Product line,Electronic accessories,Fashion accessories,Food and beverages,Health and beauty,Home and lifestyle,Sports and travel
Branch,,,,,,
A,60,51,58,47,65,59
B,55,62,50,53,50,62
C,55,65,66,52,45,45


Очікуваний результат:

![](https://drive.google.com/uc?export=view&id=1rueAdko6S3UxIHGtojetTxlES-EyM6Yb)

3.2. Викристовуючи змінну `invoices_by_category` дайте відповідь програмно (тобто значення треба не просто знайти очима, а вивести за допомогою коду), в якому філіалі магазину (`Branch`) найбільше інвойсів із покупкою товарів категорії "Електронні аксесуари" (`Electronic accessories`)?


In [29]:
max_invoices_count = invoices_by_category['Electronic accessories'].max()
print(f"Кількість інвойсів: {max_invoices_count}")

Кількість інвойсів: 60


4-6. **Творче завдання на розвиток аналітичного мислення**

Крок 1. Сформулюйте ТРИ питання (гіпотези) до наявних даних, які допомогли б вам зрозуміти, які користувачі що, де та коли найбільше/найменше купують, аби дати на основі цих гіпотез рекомендації бізнесу. Звісно питання мають бути не тими, на які ми вже відповіли в завданнях модулю.

Крок 2. Знайдіть відповіді на свої питання з допомогою функціоналу pandas.

Крок 3. Напишіть, як відповідь на це питання може бути використана для прийняття бізнес рішень.   
   
 Питання можуть бути будь-якої складності, але їх має бути 3. Кожне питання оцінюється як 1 завдання. Без виконання цього завдання ДЗ не приймається. Якщо є питання щодо виконання - пишіть у чат 🙌


### Питання (Гіпотези)
##### Питання 1
###### Чи відрізняється середній чек постійних клієнтів від нових у різних містах?
###### Гіпотеза: У деяких містах лояльні клієнти витрачають значно більше, ніж нові.
##### Питання 2
###### Яка категорія товарів має найвищий середній прибуток у вечірні години (після 18:00) у філії B?
###### Гіпотеза: У вечірній час попит може зсуватись на певні категорії — це можна використати для таргетованих промоакцій.
##### Питання 3
###### Яка група клієнтів (за статтю та типом) найчастіше купує великі обсяги (кількість товарів > середнього) у вихідні дні?
###### Гіпотеза: Можливо, існує сегмент клієнтів, який робить «оптові» покупки саме у вихідні.

### Відповіді на Питання (Гіпотези):

In [30]:
# Відповідь на Питання 1
avg_total = df.groupby(['City', 'Customer type'])['Total'].mean().reset_index()
avg_total

,City,Customer type,Total
0,Mandalay,Member,325.482945
1,Mandalay,Normal,314.329257
2,Naypyitaw,Member,336.575636
3,Naypyitaw,Normal,337.656755
4,Yangon,Member,321.182488
5,Yangon,Normal,303.831763


In [31]:
# Відповідь на Питання 2
df['Hour'] = pd.to_datetime(df['Time'], format="%H:%M").dt.hour
evening_branch_b = df[(df['Hour'] >= 18) & (df['Branch'] == 'B')]
profit_by_product = (evening_branch_b.groupby('Product line')['gross income'].mean().reset_index().sort_values(by='gross income', ascending=False))
profit_by_product

,Product line,gross income
5,Sports and travel,17.592967
4,Home and lifestyle,17.303321
3,Health and beauty,16.057727
0,Electronic accessories,15.942783
1,Fashion accessories,11.318500
2,Food and beverages,10.702316


In [32]:
# Відповідь на Питання 3
df['Date'] = pd.to_datetime(df['Date'])
df['Weekday'] = df['Date'].dt.dayofweek
weekends = df[df['Weekday'] >= 5]
mean_quantity = df['Quantity'].mean()
group = weekends[weekends['Quantity'] > mean_quantity]
result = group.groupby(['Gender', 'Customer type'])['Invoice ID'].count().reset_index(name='Count')
result

,Gender,Customer type,Count
0,Female,Member,43
1,Female,Normal,42
2,Male,Member,39
3,Male,Normal,33


### Відповіді на Питання (Гіпотези) по результатам виконання коду. Рекомендації бізнесу
##### Питання 1
###### У Mandalay та Yangon постійні покупці витрачають більше, ніж нові. Тому варто для цих міст підтримувати та розвивати програми лояльності.
###### У Naypyitaw варто переглянути стратегію лояльності; з'ясувати, чому нові клієнти витрачають більше: можливо, потрібен окремий welcome-бонус, а постійних клієнтів стимулювати додатковимии сервісами, товарами.

##### Питання 2
###### Зосередити вечірні промоакції та активації саме на категоріях Sports and Travel, Home and Lifestyle та Health and beauty у філії В, які мають найвищий середній прибуток у вечірні години. У години пік (18:00) виставляти ці товари у найбільш видимих зонах магазину.
###### Для категорій із найменшою вечірнею маржею можна спробувати знижки або кросс-продаж у комбінації з топовими товарами, щоб збільшити середній чек.

##### Питання 3
###### Найчастіше великі покупки у вихідні роблять жінки-постійні клієнти та жінки-нові клієнти (2 місце).
###### Створити спеціальні «вихідні пакети» чи набори для жінок (особливо для постійних клієнток) — це головна аудиторія великих чеків у суботу-неділю. Для постійних клієнток запропонувати додаткові бонуси за великі покупки у вихідні — наприклад, кешбек, подарункові товари чи сертифікати.
###### Для жінок-нових клієнток, які теж активно роблять великі закупи, запустити програму лояльності, щоб перевести їх у категорію Member.
###### Для чоловіків — протестувати персоналізовані пропозиції або комбо-набори, щоб збільшити їх обсяг покупок.
###### Налаштувати маркетингові розсилки чи push-повідомлення у п’ятницю — мотивувати прийти на «великі вихідні закупи».